# Web Scraping with Python

Feng Li

School of Statistics and Mathematics

Central University of Finance and Economics

[feng.li@cufe.edu.cn](mailto:feng.li@cufe.edu.cn)

[https://feng.li/python](https://feng.li/python)

# What Is Web Scraping?

The automated gathering of data from the internet is nearly as old as the internet itself. Although web scraping is not a new term, in years past the practice has been more commonly known as screen scraping, data mining, web harvesting, or similar variations. General consensus today seems to favor web scraping, so that is the term I use throughout the book, although I also refer to programs that specifically traverse multiple pages as web crawlers or refer to the web scraping programs themselves as bots.


In theory, web scraping is the practice of gathering data through any means other than a program interacting with an API (or, obviously, through a human using a web browser). This is most commonly accomplished by writing an automated program that queries a web server, requests data (usually in the form of HTML and other files that compose web pages), and then parses that data to extract needed information.

In practice, web scraping encompasses a wide variety of programming techniques and technologies, such as data analysis, natural language parsing, and information security. Because the scope of the field is so broad, this book covers the fundamental basics of web scraping and crawling in Part I and delves into advanced topics in Part II. I suggest that all readers carefully study the first part and delve into the more specific in the second part as needed.

# Your First Web Scraper

## Let's try the toy first

In [1]:
from urllib.request import urlopen
html = urlopen('https://feng.li/python/')
print(html.read())

b'<!doctype html>\n<html lang="en-US" class="respect-color-scheme-preference">\n<head>\n\t<meta charset="UTF-8" />\n\t<meta name="viewport" content="width=device-width, initial-scale=1" />\n\t<title>Python\xe7\xa8\x8b\xe5\xba\x8f\xe8\xae\xbe\xe8\xae\xa1 &#8211; Dr. Feng Li</title>\n<meta name=\'robots\' content=\'max-image-preview:large\' />\n<link rel=\'dns-prefetch\' href=\'//s.w.org\' />\n<link rel="alternate" type="application/rss+xml" title="Dr. Feng Li &raquo; Feed" href="https://feng.li/feed/" />\n<link rel="alternate" type="application/rss+xml" title="Dr. Feng Li &raquo; Comments Feed" href="https://feng.li/comments/feed/" />\n\t\t<script>\n\t\t\twindow._wpemojiSettings = {"baseUrl":"https:\\/\\/s.w.org\\/images\\/core\\/emoji\\/13.1.0\\/72x72\\/","ext":".png","svgUrl":"https:\\/\\/s.w.org\\/images\\/core\\/emoji\\/13.1.0\\/svg\\/","svgExt":".svg","source":{"concatemoji":"https:\\/\\/feng.li\\/wordpress\\/wp-includes\\/js\\/wp-emoji-release.min.js?ver=5.8.2"}};\n\t\t\t!function

The above doesn’t look so great. Below is better.

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('https://feng.li/python/')
bs = BeautifulSoup(html.read(), 'html.parser')
print(bs)

<!DOCTYPE html>

<html class="respect-color-scheme-preference" lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>Python程序设计 – Dr. Feng Li</title>
<meta content="max-image-preview:large" name="robots"/>
<link href="//s.w.org" rel="dns-prefetch"/>
<link href="https://feng.li/feed/" rel="alternate" title="Dr. Feng Li » Feed" type="application/rss+xml"/>
<link href="https://feng.li/comments/feed/" rel="alternate" title="Dr. Feng Li » Comments Feed" type="application/rss+xml"/>
<script>
			window._wpemojiSettings = {"baseUrl":"https:\/\/s.w.org\/images\/core\/emoji\/13.1.0\/72x72\/","ext":".png","svgUrl":"https:\/\/s.w.org\/images\/core\/emoji\/13.1.0\/svg\/","svgExt":".svg","source":{"concatemoji":"https:\/\/feng.li\/wordpress\/wp-includes\/js\/wp-emoji-release.min.js?ver=5.8.2"}};
			!function(e,a,t){var n,r,o,i=a.createElement("canvas"),p=i.getContext&&i.getContext("2d");function s(e,t){var a=String.fromCharCode;p.cl

## The complete case

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('https://feng.li/python/')
bs = BeautifulSoup(html.read(), 'html.parser')
nameList = bs.findAll('div', {'class':'entry-content'})
for name in nameList:
    print(name.get_text())




Contents1 课程简介2 授课教师3 参考书4 讲课视频5 幻灯片
课程简介
Python程序设计是面向财经和统计专业学生开设的一门以应用为主的编程课程，该课程最早由李丰老师在中央财经大学以公开讲座的形式开设，后成为中央财经大学金融、会计和MBA项目的核心课程。
授课教师


李丰博士现任中央财经大学统计与数学学院副院长、副教授、硕士生导师。博士毕业于瑞典斯德哥尔摩大学，研究领域包括贝叶斯统计学，预测方法，大数据分布式学习等。曾获瑞典皇家统计学会 Cramér 奖，国际贝叶斯学会青年奖励基金， 第二届全国高校经管类实验教学案例大赛二等奖。主持和参与多项国家自然科学基金项目。
李丰博士最新研究成果发表在统计期刊 Journal of Computational and Graphical Statistics，Journal of Business and Economic Statistics, Statistical Analysis and Data Mining，经济与管理学期刊 International Journal of Forecasting，Journal of Business Research，运筹学期刊European Journal of Operational Research, Journal of the Operational Research Society，人工智能期刊 Expert Systems with Applications，医学期刊 BMJ Open, Journal of Surgical Research, Journal of Affective Disorders等。同时著有 Bayesian Modeling of Conditional Densities，《大数据分布式计算与案例》和《统计计算》。


参考书
Python可以被广泛地使用在财经领域，以下列出一些零基础书目。
类别书名中译本数据分析Python for Data Analysis (by Wes McKinney)利用Python进行数据分析（原书第2版）数据抓取Web Scraping with Python: Collecting More Data from the Modern Web (by Ryan Mitchel

## Web Scraping with `BeautifulSoup`

Let's start with this page

https://finance.eastmoney.com/a/cgnjj_1.html

In [1]:
import logging
import requests
import sys
import urllib

from bs4 import BeautifulSoup
from collections import OrderedDict
from urllib.parse import urlencode

page = 1

# Set a User agent to tell the remote we are human not machines
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:93.0) Gecko/20100101 Firefox/93.0'}

href = 'https://finance.eastmoney.com/a/cgnjj_%s.html' %page
html = requests.get(href,headers=headers)

In [2]:
# Check the request headers
html.request.headers

{'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:93.0) Gecko/20100101 Firefox/93.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [3]:
# Check the html status
html.status_code

200

In [4]:
# Parsing html
soup = BeautifulSoup(html.content, 'html.parser')
soup


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<!--published at 2021/11/25 20:14:57 by www.eastmoney.com ZP NEWS 52-->
<html>
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="webkit" name="renderer"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>国内经济 _ 东方财富网</title>
<link href="Style/Layout?v=D-Lx7AIA9yzpTaxrfzY510uaHfFcF-f4wjxjND2x6AM1" rel="stylesheet" type="text/css"/>
<link href="Style/Module/ModuleStyle?v=Hg8E__Husi8eCaIDYam-IlV9PhIrIxKthXakK1TZGko1" rel="stylesheet" type="text/css"/>
<link href="Style/List?v=eGUa4FK6efrUwjDP3ziCyzxGwcRa659KOU2VShvbQco1" rel="stylesheet" type="text/css"/>
<link href="favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<base target="_blank"/>
</head>
<body style="margin-top:43px">
<div style="background-color:#fff;width:1000px;margin:0 auto;">
<img id="weixin-share" src="//cmsjs.eastmoney.com/common/wei

In [5]:
divs = soup.findAll('ul', {"id": "newsListContent"})
divs

[<ul id="newsListContent">
 <li id="newsTr0">
 <!--图-->
 <div class="image">
 <a href="http://finance.eastmoney.com/a/202111252193445986.html" target="_blank">
 <img src="//dfscdn.dfcfw.com/download/D25405819968691554918.jpg">
 </img></a>
 </div>
 <!--文-->
 <div class="text">
 <p class="title">
 <a href="http://finance.eastmoney.com/a/202111252193445986.html" target="_blank">
                     美商务部将12家中国企业列入“实体清单” 商务部：将向美方进行严正交涉
                 </a>
 </p>
 <p class="info" title="【美商务部将12家中国企业列入“实体清单” 商务部：将向美方进行严正交涉】在今天（11月25日）商务部举行的例行新闻发布会上，有媒体问，美国商务部24日宣布，12家中国企业因不符合美国国家安全利益或外交政策被列入“实体清单”。商务部对此有何回应？商务部新闻发言人束珏婷表示，美国商务部出台新的制裁清单，不符合中美两国元首共识，不利于中美两国，不利于全球产业链供应链安全和世界经济复苏。美方泛化国家安全概念，随意出台制裁措施，严重缺乏事实依据，程序非常不透明，中方表示强烈抗议，将向美方进行严正交涉。（央视）">
                     【美商务部将12家中国企业列入“实体清单” 商务部：将向美方进行严正交涉】在今天（11月25日）商务部举行的例行新闻发布会上，有媒体问，美国商务部24日宣布，1...
                 </p>
 <p class="time">
                 11月25日 15:26
             </p>
 </div>
 <!--分享-->
 <!--
         <div class="share">
         

In [30]:
divs = soup.findAll('div', {"class": "text text-no-img"})
divs

[<div class="text text-no-img">
 <p class="title">
 <a href="http://finance.eastmoney.com/a/202111252193692572.html" target="_blank">
                     由“封盘”到“免赎回费” 幻方量化又抛“重磅炸弹” 最新回应：市场变化 阶段性缩减管理规模
                 </a>
 </p>
 <p class="info" title="在宣告“封盘”十天后，头部量化私募幻方量化再度公告，即日起免除所有已发行人民币基金的赎回费用。对此，幻方量化回应财联社记者称，目前，市场环境发生了变化，未来一段时间量化策略可能面对比较复杂的市场环境，公司打算阶段性缩减管理规模，同时也降低投资者调整基金配置的成本，这有利于管理人和投资者快速应对市场的变化。">
                     在宣告“封盘”十天后，头部量化私募幻方量化再度公告，即日起免除所有已发行人民币基金的赎回费用。对此，幻方量化回应财联社记者称，目前，市场环境发生了变化，未来一段时间量化策略可能面对比较复杂的市场环境，...
                 </p>
 <p class="time">
                 11月25日 20:00
             </p>
 </div>,
 <div class="text text-no-img">
 <p class="title">
 <a href="http://finance.eastmoney.com/a/202111252193675764.html" target="_blank">
                     是否加大了房地产开发贷款投放？重庆2家上市银行回应
                 </a>
 </p>
 <p class="info">
                     11月25日，渝农商行、重庆银行在“重庆辖区2021年投资者网上集体接待日”活动上被提问“最近是否按照监管要求加大了房地产开发贷投放”。
                 </p>
 <p class="time">
            

In [55]:
import csv
newsData =  open("data/newsData.csv", 'w')
csv_writer = csv.writer(newsData, delimiter="\001")
for div in divs:
    # News title
    titleinfo = div.find('a')
    title = titleinfo.get_text().strip()
    # News url
    url = titleinfo['href']
    # News abstract
    abstract = div.find('p', {"class": "info"}).get_text().strip()
    # Time
    time = div.find('p', {"class": "time"}).get_text().strip()
    print([title, time, abstract, url])
    csv_writer.writerow([title, time, abstract, url])
newsData.close()


['由“封盘”到“免赎回费” 幻方量化又抛“重磅炸弹” 最新回应：市场变化 阶段性缩减管理规模', '11月25日 20:00', '在宣告“封盘”十天后，头部量化私募幻方量化再度公告，即日起免除所有已发行人民币基金的赎回费用。对此，幻方量化回应财联社记者称，目前，市场环境发生了变化，未来一段时间量化策略可能面对比较复杂的市场环境，...', 'http://finance.eastmoney.com/a/202111252193692572.html']
['是否加大了房地产开发贷款投放？重庆2家上市银行回应', '11月25日 19:15', '11月25日，渝农商行、重庆银行在“重庆辖区2021年投资者网上集体接待日”活动上被提问“最近是否按照监管要求加大了房地产开发贷投放”。', 'http://finance.eastmoney.com/a/202111252193675764.html']
['“稳定”、“护城河”不再那么重要！橡树资本霍华德·马克斯重磅发声：这些变化将会有大影响', '11月25日 19:11', '“我关注重点并非‘小的宏观’变化，比如明年GDP、通胀和利率会发生什么变化，而是‘大的宏观’变化，这些变化将在未来很多年内对我们的生活产生影响。”近日，橡树资本创始人及联席董事长霍华德·马克斯(How...', 'http://finance.eastmoney.com/a/202111252193677532.html']
['博远基金钟鸣远：利率债近期或迎来较好配置窗口', '11月25日 19:05', '11月25日，博远基金总经理钟鸣远发表观点指出，在金融弱宽松环境下，大类资产配置应重点关注利率债、高等级信用债和成长股，利率债近期或迎来较好配置窗口。钟鸣远指出，从最新的月度经济数据来看，10月以来，...', 'http://finance.eastmoney.com/a/202111252193675132.html']
['生态环境部：温室气体自愿减排交易应避免“地方保护”政策', '11月25日 19:05', '11月25日，生态环境部应对气候变化司副司长陆新明在11月例行新闻发布会上表示，生态环境部注意到近期个别地方出台文件对风电、光伏等新能源和可再生能源项目相关碳指标进行限制，项目收益归项目所在地所有。', 'http

In [54]:
import requests
import sys 

from bs4 import BeautifulSoup


def get_body(href):
    """Function to retrieve news content given its url.

    Args:
        href: url of the news to be crawled.

    Returns:
        content: the crawled news content.

    """
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:93.0) Gecko/20100101 Firefox/93.0'}
    html = requests.get(href, headers=headers)
    soup = BeautifulSoup(html.content, 'html.parser')
    div = soup.find('div', {"id": "ContentBody"})
    paras = div.findAll('p')
    content = ''
    for p in paras:
        ptext = p.get_text().strip().replace("\n", "")
        content += ptext
    return content



if __name__ == "__main__":
    # Getting and printing content for each url in the crawled web list pages
    with open("data/newsData.csv") as f:
        for line in f:
            title, date, abstract, href = line.strip().split('\001')
            # Printing progress onto console
            print('Scraping ' + href)
            content = get_body(href)
            print('\001'.join([title, date, abstract, href, content]))

Scraping http://finance.eastmoney.com/a/202111252193692572.html
由“封盘”到“免赎回费” 幻方量化又抛“重磅炸弹” 最新回应：市场变化 阶段性缩减管理规模11月25日 20:00在宣告“封盘”十天后，头部量化私募幻方量化再度公告，即日起免除所有已发行人民币基金的赎回费用。对此，幻方量化回应财联社记者称，目前，市场环境发生了变化，未来一段时间量化策略可能面对比较复杂的市场环境，...http://finance.eastmoney.com/a/202111252193692572.html在宣告“封盘”十天后，头部量化私募幻方量化再度公告，即日起免除所有已发行人民币基金的赎回费用。对此，幻方量化回应财联社记者称，目前，市场环境发生了变化，未来一段时间量化策略可能面对比较复杂的市场环境，公司打算阶段性缩减管理规模，同时也降低投资者调整基金配置的成本，这有利于管理人和投资者快速应对市场的变化。量化多头虽然在今年大幅跑赢主观多头，但在过去的一段时间里，量化的业绩并不突出。一位百亿私募创始人分析，由于指增和中性是国内量化策略中资金容量最大的策略类型，几乎所有主流量化机构都出现了较大的回撤。另有量化私募表示，由于量化指数增强策略进行全市场选股，且持仓分散，强指数弱个股走势，使得这段时间量化策略获取超额收益的难度相当高。对于量化私募而言，接下来的超额在哪里？有业内人士称，不同的管理人有不同的做法，有的采取放开敞口拥抱波动，有的全面拥抱机器学习，从线性模型往非线性模型甚至更复杂的模型转变，也有的开始尝试与主观研究相结合。“但不管哪个方向，都离不开对更高的算力、更精细化的模型和更强的团队的投入。”阶段性缩减管理规模11月25日下午，顶尖量化私募——幻方量化发布公告称，为维护投资者利益，方便投资者调整基金配置，即日起免除所有已发行人民币基金的赎回费用。不少市场人士揣测幻方量化此举在鼓励持有人赎回。对此，幻方量化回应记者称，目前，市场环境发生了变化，未来一段时间量化策略可能面对比较复杂的市场环境，公司打算阶段性缩减管理规模，同时也降低投资者调整基金配置的成本，这有利于管理人和投资者快速应对市场的变化。就在十天之前，业绩11月15日，幻方量化曾公告，公司暂停旗下全部产品的申购(含追加)。彼时，上海证券报援引幻方量化人士

博远基金钟鸣远：利率债近期或迎来较好配置窗口11月25日 19:0511月25日，博远基金总经理钟鸣远发表观点指出，在金融弱宽松环境下，大类资产配置应重点关注利率债、高等级信用债和成长股，利率债近期或迎来较好配置窗口。钟鸣远指出，从最新的月度经济数据来看，10月以来，...http://finance.eastmoney.com/a/202111252193675132.html11月25日，博远基金总经理钟鸣远发表观点指出，在金融弱宽松环境下，大类资产配置应重点关注利率债、高等级信用债和成长股，利率债近期或迎来较好配置窗口。钟鸣远指出，从最新的月度经济数据来看，10月以来，工业生产环比增速小幅回升，消费增速低于疫情发生前水平。“我们面临的中期环境大概率是金融弱宽松局面。在投资逻辑上，这对应着利率债、高等级信用债和成长股的大类资产配置布局。”（文章来源：中国证券报·中证网）
Scraping http://finance.eastmoney.com/a/202111252193673475.html
生态环境部：温室气体自愿减排交易应避免“地方保护”政策11月25日 19:0511月25日，生态环境部应对气候变化司副司长陆新明在11月例行新闻发布会上表示，生态环境部注意到近期个别地方出台文件对风电、光伏等新能源和可再生能源项目相关碳指标进行限制，项目收益归项目所在地所有。http://finance.eastmoney.com/a/202111252193673475.html11月25日，生态环境部应对气候变化司副司长陆新明在11月例行新闻发布会上表示，生态环境部注意到近期个别地方出台文件对风电、光伏等新能源和可再生能源项目相关碳指标进行限制，项目收益归项目所在地所有。对此，需强调两点：第一，项目业主参与温室气体自愿减排交易的权益受国家法律保护，地方政府无权对项目业主参与减排量交易的正当权益进行限制或收归己有；第二，温室气体自愿减排交易是全国性交易，地方不应该出台与国家有关政策相悖的“地方保护”政策。（文章来源：中国证券报·中证网）
Scraping http://finance.eastmoney.com/a/202111252193666689.html
广西紧抓中国东盟自贸区3.0版建设契机 谋划加强与东盟合作11月25日 18:48中国

全国共办理7600余件环境损害赔偿案件 金额超90亿元11月25日 18:1411月25日，生态环境部召开11月例行新闻发布会。生态环境部法规与标准司司长别涛介绍环境法规与标准相关情况时表示，生态环境损害赔偿制度改革取得了积极成效，截至11月，涉及赔偿金额超过90亿元。别涛介绍...http://finance.eastmoney.com/a/202111252193638738.html11月25日，生态环境部召开11月例行新闻发布会。生态环境部法规与标准司司长别涛介绍环境法规与标准相关情况时表示，生态环境损害赔偿制度改革取得了积极成效，截至11月，涉及赔偿金额超过90亿元。别涛介绍，根据中办、国办《生态环境损害赔偿制度改革方案》，生态环境部积极推动地方和有关部门协同发力，所有省份和新疆生产建设兵团以及388个市地(含直辖市区、县)印发实施方案。明确了推进路径、职责分工。各地针对赔偿纠纷的磋商、调查鉴定评估和赔偿资金的使用、管理和监督，制定了共327份配套的文件，严格追究生态环境损害赔偿责任，以弥补行政处罚和行政责任追究的不足，努力破解企业造成污染、周边群众受害、最后政府买单的不合理局面。“到本月底，全国各地共办理生态环境损害赔偿案件有7600余件，涉及赔偿金额超过90亿元。”别涛说，各个地方在推进这项工作中都是以案例实践为重要的抓手来推进改革，推动有效修复了一批受损的生态环境，包括土壤、地下水、耕地、林地、草地、矿区、草原等。在生态环境损害赔偿的制度建设和立法方面，别涛表示，改革试行以来，生态环境部门联合最高法、最高检，司法部等国务院相关职能部门，积极推动国家和地方立法，规范诉讼规则，完善技术规范和赔偿资金的使用管理的途径。据介绍，民法典、长江保护法等5部法律、《中央生态环境保护督察工作规定》和13个省级生态环境保护督察办法、19个省份的地方性法规都规定了生态环境损害赔偿内容。最高法、最高检先后发布《关于审理生态环境损害赔偿案件的若干规定(试行)》《人民检察院公益诉讼办案规则(试行)》。生态环境部联合市场监管总局发布了6项生态环境损害鉴定评估技术标准，联合有关部门印发了《关于推进生态环境损害赔偿制度改革若干具体问题的意见》。为生态环境损害赔偿立法奠定了实践的基础。（文章来源：光明网）
Scraping http://finance.eastmoney.c

专访：中老铁路项目体现了中老命运共同体精神——访中国驻老挝大使姜再冬11月25日 18:10“中老铁路既是联通之路，也是友谊之路，项目合作体现了中老命运共同体精神。”中国驻老挝大使姜再冬日前在万象接受新华社记者专访时如是说。谈及中老铁路项目合作的特点和启示，姜再冬认为中老铁路是双方在高层引领...http://finance.eastmoney.com/a/202111252193638181.html“中老铁路既是联通之路，也是友谊之路，项目合作体现了中老命运共同体精神。”中国驻老挝大使姜再冬日前在万象接受新华社记者专访时如是说。谈及中老铁路项目合作的特点和启示，姜再冬认为中老铁路是双方在高层引领下真诚合作、追求卓越的项目，双方要致力于通过加强互利合作，不断丰富中老命运共同体内涵。中老铁路工程于2016年12月全面开工，计划2021年12月建成通车。姜再冬表示，这是“一带一路”倡议同老挝“陆锁国变陆联国”战略对接取得的重大成果。作为中老友谊的标志性项目，这条铁路寄托着两国人民对美好生活的向往，展现了中老双方团结合作的力量。项目建设过程中许多人物和事件都可圈可点。姜再冬说，中老铁路建设始终得到双方高层有力指导和推动。在双方高层战略指引下，两国各有关部门和地方密切配合、特事特办，开通人员“快捷通道”和货物“绿色通道”，确保项目建设在疫情下顺利推进。提到中老铁路的建设，姜再冬表示，中老铁路所经过的地质地形复杂，仅老挝段正线桥隧比就超过62%。广大建设者克服自然条件限制，高标准建设、高质量建成，确保工程质量优于同类铁路，不仅为早日通车也为长远运营打下坚实基础。姜再冬表示，中老铁路既是中老合作的重点项目，也是地区路网的关键枢纽，对提升中老互利合作和地区互联互通都有重要意义。姜再冬认为，双方企业要力争明年早些时候完成老泰铁路换装工程，并进而研究启动中老泰连接线建设，以便南下同马新铁路网联通、北上同中欧班列对接，真正实现翻山越岭、通江达海，把老挝的“陆锁”短板转化为“枢纽”优势，双方还要加快铁路沿线综合开发，为老挝经济恢复和中老务实合作注入动力，同时加大服务民生力度，在后续运营和开发过程中，双方要继续为普通民众带来更多看得见、摸得着的利益，以实际行动体现中老命运共同体精神。对于同老挝的合作，姜再冬表示，“我们不仅同老方建设老挝第一条现代化铁路，还已建成第一条高速公路，推

## Web Crawling with `Scrapy`*

One of the challenges of writing web crawlers is that you’re often performing the same tasks again and again: find all links on a page, evaluate the difference between internal and external links, go to new pages. These basic patterns are useful to know and to be able to write from scratch, but the Scrapy library handles many of these details for you.

###  Installing Scrapy

- After Anaconda is installed, you can install Scrapy by using this command:
   
      conda install -c conda-forge scrapy

### Dealing with Different Website Layouts

Fortunately, in most cases of web crawling, you’re not looking to collect data from sites you’ve never seen before, but from a few, or a few dozen, websites that are pre-selected by a human. This means that you don’t need to use complicated algorithms or machine learning to detect which text on the page “looks most like a title” or which is probably the “main content.” You can determine what these elements are manually.

The most obvious approach is to write a separate web crawler or page parser for each website. Each might take in a URL, string, or BeautifulSoup object, and return a Python object for the thing that was scraped.


## Initializing a New Spider

To create a new spider in the current directory, run the following from the **command line (NOT THE PYTHON PROMPT)**:
```
    scrapy startproject wikiSpider
```    
    
This creates a new subdirectory in the directory the project was created in, with the title wikiSpider. Inside this directory is the following file structure:

- scrapy.cfg
- wikiSpider
  - spiders
     - __init.py__
  - items.py
  - middlewares.py
  - pipelines.py
  - settings.py
  - __init.py__

### Generate some spiders with templates from the command line

    scrapy genspider example example.com 
    scrapy genspider example2 example.com 
    scrapy genspider example3 example2.com 

### Writing a Simple Scraper

To create a crawler, you will add a new file inside the spiders directory at wikiSpider/wikiSpider/spiders/article.py. In your newly created **article.py** file, write the following:

```python
    import scrapy

    class ArticleSpider(scrapy.Spider):
        name='article'

        def start_requests(self):
            urls = [
                'http://en.wikipedia.org/wiki/Python_%28programming_language%29',
                'https://en.wikipedia.org/wiki/Functional_programming',
                'https://en.wikipedia.org/wiki/Monty_Python']
            return [scrapy.Request(url=url, callback=self.parse) for url in urls]

        def parse(self, response):
            url = response.url
            title = response.css('h1::text').extract_first()
            print('URL is: {}'.format(url))
            print('Title is: {}'.format(title))
```

### Run this article spider

You can run this article spider by navigating to the wikiSpider/wikiSpider directory and running from the command line:

    scrapy runspider article.py
        
### Run your project with at the project root directory

    scrapy crawl table -o table.csv  --logfile table.log
    

### Scrapy Shell

To do the crawler interactively, just run from the command line

```bash
scrapy shell "http://en.wikipedia.org/wiki/Python_%28programming_language%29"
```

# Lab 

Use `scrapy` framework to implement the we studied with `BeautifulSoup`